In [48]:
'''In this notebook we will download all of the IBL video files and then compress them to 64x? pixels and 30fps
  then we will save them to a local folder from which we can run an http server'''
import numpy as np
import urllib.request


In [ ]:
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')

In [28]:
# Get all eid + probe combinations with timing info, wheel info, spikes, clusters, and channels.
eid_info_map = np.load("/mnt/c/proj/IBL/filtered_eids/brainwide_eid_info_map.npy", allow_pickle=True).item()

good_eids = {}
for eid in eid_info_map:
  eid_info = eid_info_map[eid]

  if eid_info["timing_info"] and eid_info["wheel_info"]:
    good_probes = []
    for probe in ['probe00', 'probe01']:
      probe_info = eid_info[probe]
      if probe_info["spikes"] and probe_info["clusters"] and probe_info["channels"]:
        good_probes.append(probe)
    if len(good_probes) > 0:
      good_eids[eid] = good_probes
      
eids = list(good_eids.keys())

In [30]:
eids[0]

'02fbb6da-3034-47d6-a61b-7d06c796a830'

In [ ]:
# request = urllib2.Request(url)
# base64string = base64.b64encode('%s:%s' % (username, password))
# request.add_header("Authorization", "Basic %s" % base64string)   
# result = urllib2.urlopen(request)

In [49]:
dsets = one.type2datasets(eid, '_iblrig_Camera.raw', details=True)
videos = ['left','right','body']

opener = urllib.request.build_opener()
opener.addheaders[]

for video in videos:
  dset = next(d for idx, d in dsets.iterrows() if video in d['rel_path'])
  url = one.record2url(dset)
  print(url)
  urllib.request.urlretrieve(url, "flatironVideos/"+video+".mp3")
  # also get the timestamps
  # To get timestamps for video data so you can find out what times each frame is at
  dsets = one.type2datasets(eid, 'camera.times')
  dset = next(d for d in dsets if video in d)
  # load data
  times = one.load_dataset(eid, dset)
  np.save('flatironVideos/'+video+"_times.npy",times)

https://ibl.flatironinstitute.org/wittenlab/Subjects/ibl_witten_26/2021-01-27/002/raw_video_data/_iblrig_leftCamera.raw.99d8296f-e3e6-402d-a74f-52dec8ce0467.mp4


HTTPError: HTTP Error 401: Unauthorized

In [ ]:
# STREAM VIDEO DATA
from ibllib.io.video import get_video_frames_preload, get_video_meta
from one.api import ONE
import numpy as np
one = ONE()

eid = '73918ae1-e4fd-4c18-b132-00cb555b1ad2'
video = 'left'

# A couple of example ways to get video URL

# Using cache
# This gives a pd dataframe of the details of raw video files
dsets = one.type2datasets(eid, '_iblrig_Camera.raw', details=True)
# Find the one that corresponds to video='left'
dset = next(d for idx, d in dsets.iterrows() if video in d['rel_path'])
# Get the url of the data for first dset
url = one.record2url(dset)


# Download first five frames of video, keeping only the first colour
# See these scripts for more detail
# (https://github.com/int-brain-lab/ibllib/blob/master/ibllib/io/video.py)
#

video = get_video_frames_preload(url, frame_numbers=range(5), mask=np.s_[:, :, 0])
# to get some video meta (if you want)
meta = get_video_meta(url)

# To get timestamps for video data so you can find out what times each frame is at
dsets = one.type2datasets(eid, 'camera.times')
dset = next(d for d in dsets if video in d)
# load data
times = one.load_dataset(eid, dset)


# STREAM EPHYS DATA
from viewspikes.data import stream
pid = 'af2a0072-e17e-4368-b80b-1359bf6d4647'
# Stream 3 seconds of ap data from 100s
ap, dsets, t0 = stream(pid, t=100, one=one, cache=True, tlen=3)
# Get out the raw data
ap_data = ap[:, :-1].T

# See here for examples of filters to apply to the data
# https://github.com/int-brain-lab/iblapps/blob/develop/atlaselectrophysiology/alignment_with_easyqc.py#L128
# Also here for how to stream lf data
# https://github.com/int-brain-lab/iblapps/blob/develop/atlaselectrophysiology/alignment_with_easyqc.py#L115